# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame


In [2]:
# Load the csv exported in Part I to a DataFrame
weather_data=pd.read_csv("../WeatherPy/Output/weather_data.csv", encoding="ISO-8859-1")
weather_data

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,San QuintÃ­n,30.4833,-115.9500,59.14,51,5,5.79,MX,1610605952
1,Atuona,-9.8000,-139.0333,77.54,78,42,11.34,PF,1610605822
2,Sirpur,19.4833,79.6000,84.40,20,0,3.85,IN,1610605954
3,Codrington,-38.2667,141.9667,58.91,64,96,15.48,AU,1610605955
4,Bengkulu,-3.8004,102.2655,84.92,61,100,7.25,ID,1610605957
...,...,...,...,...,...,...,...,...,...
570,Tigil',57.8000,158.6667,-0.62,90,99,6.22,RU,1610607364
571,Vredendal,-31.6683,18.5012,80.29,46,0,3.24,ZA,1610607365
572,Kolpashevo,58.3131,82.9089,-4.58,91,100,10.56,RU,1610607366
573,Yeppoon,-23.1333,150.7333,86.00,48,0,10.36,AU,1610607367


### Humidity Heatmap


In [3]:
#Configure gmaps.
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Add Heatmap layer to map.
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_data['Humidity'].max(),
                                 point_radius=3)


# Add humidity layer
fig.add_layer(humidity_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [4]:
# View the data before setting ideal weather conditions.
weather_data.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,San QuintÃ­n,30.4833,-115.9500,59.14,51,5,5.79,MX,1610605952
1,Atuona,-9.8000,-139.0333,77.54,78,42,11.34,PF,1610605822
2,Sirpur,19.4833,79.6000,84.40,20,0,3.85,IN,1610605954
3,Codrington,-38.2667,141.9667,58.91,64,96,15.48,AU,1610605955
4,Bengkulu,-3.8004,102.2655,84.92,61,100,7.25,ID,1610605957


In [23]:
# Narrow down the cities to fit weather conditions.
#ideal weather conditions: maximum temperature between 70 to 86(F)
                         # wind speed <= 5(mph)
                         # humidity < 50 (%), cloudiness =0

idealvacation_df = weather_data.loc[(weather_data['Max temp']>=70) & (weather_data['Max temp']<=86) & 
                                    (weather_data['Humidity']<=50)& (weather_data['Cloudiness']<=0)
                                    & (weather_data['Wind Speed']<=12),: ]

# Make sure filtered location size is less than 20. 
len(idealvacation_df)

11

### Hotel Map



In [24]:
# Store into variable named `hotel_df`.
hotel_df=idealvacation_df.loc[:,["City", "Country","Lat","Lng"]]
hotel_df

,City,Country,Lat,Lng
2,Sirpur,IN,19.4833,79.6000
257,Longhua,CN,23.6137,114.1919
408,ChÄndbÄli,IN,20.7833,86.7667
409,Bijapur,IN,16.8333,75.7000
428,Bria,CF,6.5423,21.9863
449,Jodhpur,IN,26.2867,73.0300
451,Calvinia,ZA,-31.4707,19.7760
466,Chabahar,IR,25.2919,60.6430
516,DwÄrka,IN,22.2394,68.9678
571,Vredendal,ZA,-31.6683,18.5012


In [25]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Sirpur,IN,19.4833,79.6000,
257,Longhua,CN,23.6137,114.1919,
408,ChÄndbÄli,IN,20.7833,86.7667,
409,Bijapur,IN,16.8333,75.7000,
428,Bria,CF,6.5423,21.9863,
449,Jodhpur,IN,26.2867,73.0300,
451,Calvinia,ZA,-31.4707,19.7760,
466,Chabahar,IR,25.2919,60.6430,
516,DwÄrka,IN,22.2394,68.9678,
571,Vredendal,ZA,-31.6683,18.5012,


In [26]:
# Set parameters to search for hotels with 5000 meters.


params = {
    "keyword": "hotel",
    "type": "lodging",
    "radius": 5000,
    "key": gkey
}
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # Hit the Google Places API for each city's coordinates.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    hotel_data=requests.get(base_url, params=params).json()
    
    # handle exceptions.
    try:
        # Store the first Hotel result into the DataFrame.
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [27]:
# View the data to make sure hotel names have been updates.
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Sirpur,IN,19.4833,79.6000,Siddharth Hotel
257,Longhua,CN,23.6137,114.1919,
408,ChÄndbÄli,IN,20.7833,86.7667,Aranya Nivas (OTDC)
409,Bijapur,IN,16.8333,75.7000,"The Fern Residency, Vijayapura"
428,Bria,CF,6.5423,21.9863,
449,Jodhpur,IN,26.2867,73.0300,"Umaid Bhawan Palace, Jodhpur"
451,Calvinia,ZA,-31.4707,19.7760,Die Hok Self Catering
466,Chabahar,IR,25.2919,60.6430,Shahanhotel
516,DwÄrka,IN,22.2394,68.9678,Lemon Tree Premier
571,Vredendal,ZA,-31.6683,18.5012,River Lodge


In [28]:
# Filter the dataframe to get rid of empty hotel name rows.
hotel_df=hotel_df.loc[hotel_df['Hotel Name']!='']
hotel_df


,City,Country,Lat,Lng,Hotel Name
2,Sirpur,IN,19.4833,79.6000,Siddharth Hotel
408,ChÄndbÄli,IN,20.7833,86.7667,Aranya Nivas (OTDC)
409,Bijapur,IN,16.8333,75.7000,"The Fern Residency, Vijayapura"
449,Jodhpur,IN,26.2867,73.0300,"Umaid Bhawan Palace, Jodhpur"
451,Calvinia,ZA,-31.4707,19.7760,Die Hok Self Catering
466,Chabahar,IR,25.2919,60.6430,Shahanhotel
516,DwÄrka,IN,22.2394,68.9678,Lemon Tree Premier
571,Vredendal,ZA,-31.6683,18.5012,River Lodge
573,Yeppoon,AU,-23.1333,150.7333,Yeppoon Beach House


### Plot markers on top of the heatmap.

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add hotels layer ontop of heat map
fig = gmaps.figure()
hotels = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotels)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Add hotel layer and humidity layer to create a combined plot.
fig = gmaps.figure()

fig.add_layer(humidity_layer)
fig.add_layer(hotels)

fig

Figure(layout=FigureLayout(height='420px'))